In [1]:
import numpy as np 
import pandas as pd 
import os
import itertools

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Input, merge, UpSampling2D, Cropping2D, ZeroPadding2D, Reshape, core, Convolution2D, Conv2DTranspose, Concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from keras import optimizers
from keras import backend as K
from keras.optimizers import SGD
from keras.layers.merge import concatenate
from keras.preprocessing.image import array_to_img
from keras.preprocessing.image import load_img

from tensorflow.python.keras.callbacks import TensorBoard

from sklearn.metrics import fbeta_score
from sklearn.model_selection import train_test_split

from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt

import preprocess
from model_02 import get_model
import pickle
import splitdata

import keras.backend as K
import tensorflow as tf

from keras.models import load_model
import losses
import imageio

Using TensorFlow backend.


In [2]:
files = preprocess.get_split_files("data/split")
print(len(files))

Finding files in 'data/split'
data/split/41_2_x.png 
 data/split/41_2_y.png
data/split/31_3_x.png 
 data/split/31_3_y.png
data/split/4_1_x.png 
 data/split/4_1_y.png
data/split/27_4_x.png 
 data/split/27_4_y.png
data/split/16_5_x.png 
 data/split/16_5_y.png
data/split/30_3_x.png 
 data/split/30_3_y.png
data/split/36_7_x.png 
 data/split/36_7_y.png
data/split/5_0_x.png 
 data/split/5_0_y.png
data/split/36_9_x.png 
 data/split/36_9_y.png
data/split/18_3_x.png 
 data/split/18_3_y.png
data/split/28_9_x.png 
 data/split/28_9_y.png
data/split/26_8_x.png 
 data/split/26_8_y.png
data/split/25_9_x.png 
 data/split/25_9_y.png
data/split/28_12_x.png 
 data/split/28_12_y.png
data/split/1_1_x.png 
 data/split/1_1_y.png
data/split/4_4_x.png 
 data/split/4_4_y.png
data/split/28_2_x.png 
 data/split/28_2_y.png
data/split/6_2_x.png 
 data/split/6_2_y.png
data/split/36_3_x.png 
 data/split/36_3_y.png
data/split/25_7_x.png 
 data/split/25_7_y.png
data/split/26_11_x.png 
 data/split/26_11_y.png
data/split

In [3]:
np.random.shuffle(files)
files = files[0:]
n_valid = int(np.ceil(len(files) * 0.15))
n_test = int(np.ceil((len(files) - n_valid)* 0.15))
n_train = len(files) - n_valid - n_test
f_train = files[0:n_train]
f_valid = files[n_train: n_train + n_valid]
f_test = files[n_train + n_valid:]

In [4]:
print(len(f_train))
print(len(f_test))
print(len(f_valid))

274
49
58


In [5]:
X_train, Y_train = preprocess.load_split_data(f_train)
X_valid, Y_valid = preprocess.load_split_data(f_valid)

Loading data...
samples    274
Img size  ( 512, 512)
size      1.15 GB
Loading data...
samples     58
Img size  ( 512, 512)
size      0.24 GB


In [6]:
Y_train.shape

(274, 512, 512, 1)

In [7]:
n_total = 274 * 512 * 512
n_ones = np.sum(Y_train)
print(n_total)
print(n_ones)
print(n_ones / n_total)
print(1 - n_ones / n_total)
0.0029 + 0.9971

71827456
210218.0
0.0029267081379
0.997073291862


1.0

In [8]:
print(X_train.shape)
print(Y_train.shape)

(274, 512, 512, 3)
(274, 512, 512, 1)


In [9]:
model = get_model(img_size=512, n_kernels=1)
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
conv_1_1 (Conv2D)               (None, 512, 512, 1)  28          input_1[0][0]                    
__________________________________________________________________________________________________
conv_2_2 (Conv2D)               (None, 512, 512, 1)  10          conv_1_1[0][0]                   
__________________________________________________________________________________________________
pool_1 (MaxPooling2D)           (None, 256, 256, 1)  0           conv_2_2[0][0]                   
__________________________________________________________________________________________________
conv2_1 (C

/home/matthias/projects/bio-unet/model_02.py:87: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  model = Model(input=inputs, output=conv10)


In [10]:
model.compile(loss="categorical_crossentropy",
              optimizer=optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.98, nesterov=True),
              metrics=['accuracy'])
#K.get_session().run(tf.global_variables_initializer())
#tensorboard = TensorBoard(log_dir="logs") 
callbacks = [
             EarlyStopping(monitor='val_loss',
                           patience=4,
                           verbose=1),
             ModelCheckpoint("model07.h5",
                             monitor='val_loss',
                             save_best_only=True,
                             verbose=1)]

In [12]:
# batch_size determines the number of samples in each mini batch
# steps_per_epoch the number of batch iterations before a training epoch is considered finished.
# validation_steps
hist = model.fit(x=X_train,
                 y=Y_train,
                 epochs=1,
                 class_weight={1: 0.99, 0: 0.01},
                 verbose=2, callbacks=callbacks,
                 validation_data=(X_valid, Y_valid))

ValueError: `class_weight` not supported for 3+ dimensional targets.

In [ ]:
training_loss = hist.history['loss']
test_loss = hist.history['val_loss']
training_acc = hist.history['acc']
test_acc = hist.history['val_acc']

epoch_count = range(1, len(training_loss) + 1)

plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, test_loss, 'b-')
plt.legend(['Training Loss', 'Validation Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show();

plt.plot(epoch_count, training_acc, 'r--')
plt.plot(epoch_count, test_acc, 'b-')
plt.legend(['Training Acc', 'Validation Acc'])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show();

In [ ]:
model.load_weights("model02.h5")

In [ ]:
X_test, Y_test = load_split_data(f_test)

In [ ]:
Y_pred = model.predict(X_test, batch_size=1, verbose=1)
Y_round = np.around(Y_pred)

for i in range(Y_pred.shape[0]):
    preprocess.show_ndimg(X_test[i], Y_test[i])
    preprocess.show_ndimg(Y_pred[i], Y_round[i])

In [ ]:
n = 10
Y_pred = model.predict(X_train[0:n], batch_size=1, verbose=1)
Y_round = np.around(Y_pred)

for i in range(Y_pred.shape[0]):
    preprocess.show_ndimg(X_train[i], Y_train[i])
    preprocess.show_ndimg(Y_pred[i], Y_round[i])

In [ ]:

n_total = np.prod(Y_train[0].shape)
n_ones = np.sum(Y_train[0])
n_zeros = n_total - n_ones
print(n_total)
print(n_ones)
print(n_zeros)
w_zero = n_ones / n_total
w_ones = n_zeros / n_total
print(w_zero)
print(w_ones)
print(w_zero + w_ones)
print(1 - 0.0022)

In [ ]:
Y_train[0]

In [ ]:
for i in range(Y_pred.shape[0]):
    preprocess.show_ndimg(X_test[i], Y_test[i])
    preprocess.show_ndimg(Y_pred[i], Y_round[i])